In [ ]:
import torch
from ase.io import read

import schnetpack as spk
from schnetpack import properties
from schnetpack.interfaces.ase_interface import AtomsConverter
from schnetpack.interfaces.batchwise_optimizer import TorchLBFGS

In [ ]:
input_structure_file = "/home/jonas/Documents/schnetpack/tests/testdata/md_ethanol.xyz"
model_path = "/home/jonas/Documents/schnetpack/tests/testdata/md_ethanol.model"

In [ ]:
# set device
device = torch.device("cuda")

# load model
model = torch.load(model_path, map_location=device)
# remove response modules (to avoid differentiating more than once)
model.model_outputs = ["energy"]
model.do_postprocessing = False
model.required_derivatives = []
model.output_modules = torch.nn.ModuleList([model.output_modules[0]])

In [ ]:
# define neighbor list
cutoff = model.representation.cutoff.item()
nbh_list=spk.transform.MatScipyNeighborList(cutoff=cutoff)

# build atoms converter
atoms_converter = AtomsConverter(
    neighbor_list=nbh_list,
    device=device,
)

In [ ]:
ats = read(input_structure_file, index=":")

In [ ]:
# define structure mask for optimization (True for fixed, False for non-fixed)
n_atoms = len(ats[0].get_atomic_numbers())
single_structure_mask = [False for _ in range(n_atoms * 3)]
# expand mask by number of input structures (fixed atoms are equivalent for all input structures)
mask = single_structure_mask * len(ats)

In [ ]:
# convert atoms object to schnet batch
inputs = atoms_converter(ats)

In [ ]:
# Initialize optimizer
optimizer = TorchLBFGS(
    model=model,
    model_inputs=inputs,
    fixed_atoms_mask=mask
)

# run optimization
optimizer.run(fmax=0.0005, max_opt_steps=1000)

In [ ]:
# get new atomic positions
optimizer.get_relaxed_structure()